In [1]:
import requests, json, csv

In [2]:
#Словарь с данными для API
api = {
    'gateway': 'https://manifesto-project.wzb.eu/tools/',
    'key': '?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0'
}

In [23]:
"""
    Обертка для обращений к API
    Аргументы:
        string : api_method – Название метода (https://manifestoproject.wzb.eu/information/documents/api)
        *string : params – Параметры вызова
"""
def api_request(api_method, *params):
    parameters = ''
    if params:
        for i in params:
            parameters += ('&' + i)
    request_url = api['gateway'] + api_method + '.json' + api['key'] + parameters
    print(request_url)
    return json.loads(requests.get(request_url).text)

"""
    Пишет CSV в папку datasheets
    Аргументы:
        list : data – Список данных для записи
        string : name – Имя файла с расширением
"""
def write_csv(data, name):
    path = 'datasheets/' + name
    with open(path, "w", newline='') as csv_file:
            writer = csv.writer(csv_file, delimiter=',')
            for line in data:
                writer.writerow(line)

"""
    Читает из CSV данные из выбранных колонок
    Аргументы:
        string : name – Имя файла с расширением
        *string : headings – Названия колонок
        (Optional) bool : write_first_row – Записывать ли в список заголовки колонок
"""
def csv_to_list(name, *headings, write_first_row=True):
    with open("datasheets/" + name) as file_obj:
        reader = csv.DictReader(file_obj, delimiter=',')
        result = []
        if write_first_row:
            first_row = []
            for heading in headings:
                first_row.append(heading)
            result.append(first_row)
        for line in reader:
            parsed = []
            for heading in headings:
                parsed.append(line[heading])
            result.append(parsed)
        return result

In [ ]:
api_request('api_get_core', 'key=MPDS2017b')

In [15]:
api_request('api_metadata', 'keys[]=51320_197402', 'version=2017-2')

https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=51320_197402&version=2017-2


{'items': [{'party_id': 51320,
   'election_date': '197402',
   'language': 'english',
   'source': 'CEMP',
   'has_eu_code': 'FALSE',
   'is_primary_doc': 'TRUE',
   'may_contradict_core_dataset': 'FALSE',
   'manifesto_id': '51320_197402',
   'md5sum_text': '8d31d40aa571551034bb75235743cd23',
   'url_original': '/down/originals/51320_1974.pdf',
   'md5sum_original': 'CURRENTLY_UNAVAILABLE',
   'annotations': False,
   'handbook': None,
   'is_copy_of': None,
   'title': 'Let us Work Together - Labour’s Way Out of the Crisis'}],
 'missing_items': []}

In [ ]:
api_request('api_list_metadata_versions') 

In [14]:
api_request('api_texts_and_annotations', 'keys[]=41320_2009', 'version=2017-2')

https://manifesto-project.wzb.eu/tools/api_texts_and_annotations.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=41320_2009&version=2017-2


{'items': [], 'missing_items': ['41320_2009']}

In [ ]:
api_request('api_get_core_codebook', 'key=MPDS2017b')

In [24]:
core_set_from_csv = csv_to_list("core.csv", "date", "party", "id_perm", write_first_row=False)
#for i in core_set_from_csv:
    

In [17]:
core_set

[['date', 'party', 'id_perm'],
 ['194409', '11220', 'JN1LZH'],
 ['194409', '11320', 'CMR7F6'],
 ['194409', '11420', 'Z6OL6C'],
 ['194409', '11620', 'YMKVN2'],
 ['194409', '11810', 'U4SCRD'],
 ['194809', '11220', 'KU4SLH'],
 ['194809', '11320', 'SUX4JY'],
 ['194809', '11420', 'RGU9EV'],
 ['194809', '11620', 'FU1SGP'],
 ['194809', '11810', 'H326HA'],
 ['195209', '11220', '8Y82JK'],
 ['195209', '11320', 'AGKM5T'],
 ['195209', '11420', '3ENRXJ'],
 ['195209', '11620', '3ERZTJ'],
 ['195209', '11810', 'PZO59H'],
 ['195609', '11220', '8S4LJX'],
 ['195609', '11320', '87HOAY'],
 ['195609', '11420', 'CSX8VJ'],
 ['195609', '11620', 'I4XTSW'],
 ['195609', '11810', 'KKH27B'],
 ['195806', '11220', '31QUMF'],
 ['195806', '11320', 'S58Y99'],
 ['195806', '11420', 'VEMAXD'],
 ['195806', '11620', 'XMMMO1'],
 ['195806', '11810', 'YO6HUI'],
 ['196009', '11220', '5DKDUG'],
 ['196009', '11320', 'A3M93R'],
 ['196009', '11420', '461BJZ'],
 ['196009', '11620', 'YZVJX1'],
 ['196009', '11810', 'COJWYR'],
 ['196409